In [0]:
# Before executing this bash cell you need to upload all the zip files 
# gracefully prepared by me into the /content directory.
# Then execute this cell.
# It'll create a data/train relative directory and unzip the uploaded files.

%%shell
rm -r /content/data/
mkdir -p /content/data/train

for i in {0..11}
do
  unzip "/content/train$i.zip" -d /content/data/train
  unzip "/content/train0$i.zip" -d /content/data/train
done

In [0]:
import pandas as pd
from glob import glob
import os
from PIL import Image
import numpy as np
import random

In [0]:
%tensorflow_version 2.x

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

random.seed(34)

TensorFlow 2.x selected.


Using TensorFlow backend.


In [0]:
# Скачайте полные данные отсюда https://www.kaggle.com/c/dogs-vs-cats/data (примерно 820мб)
# У вас должно быть две папки train и test. Однако в данном задании мы не будем использвать test выборку
# Функцию для чтения данных мы реализовали за вас

def read_dataset(path):
    X = []
    y = []
    
    image_paths_list = glob(os.path.join(path, 'train', '*.jpg'))
    image_paths_sample = random.sample(image_paths_list, 1000)

    for image_path in image_paths_sample:
        image_name = os.path.basename(image_path)
        image_name_parts = image_name.split('.')
        label = image_name_parts[0] if len(image_name_parts) == 3 else None

        if label:
            y.append(int(label == 'cat'))
            
        
        x = image.img_to_array(image.load_img(image_path, target_size=(224, 224)))
        #x = preprocess_input(x)
            
        X.append(x)
            
    return np.array(X), np.array(y)

Перепишите код read_dataset() таким образом чтобы читать изображения пачками по 1000 и сразу применять предобученный ResNet на них, сохраняя только полученные вектора как множество X. Примените эту функцию на всём датасете 25000 изображений.

In [0]:
from tqdm import tqdm_notebook

train_directory = "data/"
batch_size = 1000

def read_dataset(path):
    X = []
    y = []
    
    image_paths_list = np.array(glob(os.path.join(path, 'train', '*.jpg')))
    
    np.random.shuffle(image_paths_list)
    
    # split into array of batch_size length arrays
    splitted_image_paths_list = np.split(image_paths_list, int(len(image_paths_list) / batch_size))
    
    for image_path_batch in tqdm_notebook(splitted_image_paths_list, desc='Batches'):
        x_batch = []
        y_batch = []
        
        for image_path in tqdm_notebook(image_path_batch, desc='Current batch', leave=False):
            image_name = os.path.basename(image_path)
            image_name_parts = image_name.split('.')
            label = image_name_parts[0] if len(image_name_parts) == 3 else None
            
            if label:
                y_batch.append(int(label == 'cat'))
            
            x = image.img_to_array(image.load_img(image_path, target_size=(224, 224)))
            x = preprocess_input(x)
            
            x_batch.append(x)
        
        x_batch = np.expand_dims(x_batch, axis=0)
        x_batch = preprocess_input(x_batch).astype(np.float16)
        
        X.append(x_batch)
        y.append(y_batch)
    
    return np.array(X), np.array(y)   

In [0]:
# Используйте функцию read_dataset чтобы получить обучающую выборку
train_directory = "data/"
features, labels = read_dataset(train_directory)

# Проверьте размерности загруженных данных
print(features.shape)
print(labels.shape)

In [0]:
# с помощью функции train_test_split поделите выборку на train и test в отношении 70/30
# В качестве признаков используйте эмбеддинги, полученные на предыдущем шаге


In [0]:
# Создаём модель для получения эмбеддингов
model = ResNet50(weights='imagenet')

# Используя объект model постройте эмбеддинги для обучающей выборки

Создайте объект XGBClassifier со стандартными параметрами

https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier

Обучите его

In [0]:
# Получите предсказания на тестовом и на обучающем множестве


In [0]:
# С помощью функции accuracy_score оцените результаты


Почитайте про параметры XGBoost здесь: https://sites.google.com/view/lauraepp/parameters
Сделайте тюнинг гиперпараметров для XGBoost:
1. Подберите максимальный num_iterations при котором скор на валидации близок к максимальному, а скорость обучения ещё приемлимая
2. Для заданного num_iterations подберите подходящий learning rate
3. С помощью библиотеки hyperopt подберите остальные гиперпараметры при фиксимрованных num_iterations и learning_rate. Не подбирайте параметр early_stopping

Оцените важность признаков с помощью поля feature_importances_

Пользуясь списком https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a посмотрите, какие классы оказались наиболее важными (со значениями importance больше 0)
    
Изменяйте параметры, чтобы оставить только действительно важные признаки